In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("hate_speech_filipino")

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4232 [00:00<?, ? examples/s]

In [11]:
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

In [12]:
print(len(train_set))
print(len(validation_set))
print(len(test_set))

10000
4232
10000
